In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../Data/bank_marketing_train.csv")
print("df.shape = " ,df.shape)
df.head()

df.shape =  (26246, 25)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,feature_1,feature_2,feature_3,feature_4,feature_5,y
0,72,retired,married,basic.4y,no,no,no,telephone,apr,tue,...,93.075,-47.1,1.453,5099.1,1.3874,73.8872,vg1,lv1,fn1,no
1,30,admin.,single,university.degree,no,yes,no,cellular,aug,tue,...,94.027,-38.3,0.886,4991.6,1.3874,31.8872,vg2,lv1,fn2,no
2,31,unemployed,married,university.degree,no,yes,no,cellular,aug,fri,...,93.444,-36.1,4.966,5228.1,4.3904,35.8872,vg3,lv1,fn3,no
3,37,admin.,married,high.school,no,yes,yes,cellular,nov,mon,...,92.649,-30.1,0.722,5017.5,1.3874,38.8872,vg4,lv1,fn4,no
4,53,unemployed,divorced,basic.9y,unknown,no,no,telephone,may,thu,...,93.994,-36.4,4.860,5191.0,2.3884,55.8872,vg5,lv1,fn5,no


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26246 entries, 0 to 26245
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             26246 non-null  int64  
 1   job             26246 non-null  object 
 2   marital         26246 non-null  object 
 3   education       26246 non-null  object 
 4   default         26246 non-null  object 
 5   housing         26246 non-null  object 
 6   loan            26246 non-null  object 
 7   contact         26246 non-null  object 
 8   month           26246 non-null  object 
 9   day_of_week     26246 non-null  object 
 10  campaign        26246 non-null  int64  
 11  pdays           26246 non-null  int64  
 12  previous        26246 non-null  int64  
 13  poutcome        26246 non-null  object 
 14  emp.var.rate    26246 non-null  float64
 15  cons.price.idx  26246 non-null  float64
 16  cons.conf.idx   26246 non-null  float64
 17  euribor3m       26246 non-null 

In [18]:
# 避免污染原始数据，先复制一个数据
df_clean = df.copy()

In [19]:
df_clean.columns.tolist()

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'y']

In [20]:
# 把999改为-1，方便后面标准化
df_clean['pdays'] = df_clean['pdays'].replace(999,-1)
df_clean['pdays'].value_counts().head()

pdays
-1     25206
 3       310
 6       288
 4        84
 12       41
Name: count, dtype: int64

In [6]:
# 观察类别不平衡情况
df_clean['y'].value_counts(normalize=True)

y
no     0.881049
yes    0.118951
Name: proportion, dtype: float64

In [7]:
unknown_counts = (df_clean == "unknown").sum()
unknown_counts[unknown_counts > 0]

job           224
marital        49
education    1154
default      5552
housing       650
loan          650
dtype: int64

In [21]:
# 把y变成0/1变量
df_clean['y'] = df_clean['y'].map({'yes': 1, 'no': 0})
df_clean['y'].value_counts()

y
0    23124
1     3122
Name: count, dtype: int64

In [9]:
# 找出数值列
numeric_cols = df_clean.select_dtypes(include=['int64','float64']).columns  #include包括，exclude删除
numeric_cols

Index(['age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'feature_1', 'feature_2', 'y'],
      dtype='object')

In [10]:
# 查看每个数值列的大致统计特征
df_clean[numeric_cols].describe()

/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,feature_1,feature_2,y
count,26246.000000,26246.00000,26246.000000,26246.000000,26246.000000,26246.000000,26246.000000,26246.000000,26246.000000,26246.0000,26246.000000,26246.000000
mean,40.154538,2.66574,-0.725863,0.181704,0.059598,93.580828,-40.491404,3.597076,5165.540197,inf,43.707477,0.118951
std,10.450543,2.87869,1.538588,0.506030,1.578188,0.586050,4.646433,1.743984,73.224175,NaN,10.850257,0.323738
min,17.000000,1.00000,-1.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000,1.3874,19.887200,0.000000
25%,32.000000,1.00000,-1.000000,0.000000,-1.800000,93.075000,-42.700000,1.334000,5099.100000,1.3874,34.887200,0.000000
50%,38.000000,2.00000,-1.000000,0.000000,1.100000,93.876000,-41.800000,4.857000,5191.000000,2.3884,41.887200,0.000000
75%,47.000000,3.00000,-1.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000,3.3894,50.887200,0.000000
max,98.000000,56.00000,25.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000,inf,99.887200,1.000000


In [22]:
# 发现feature_1有异常值，用中位数替换
df_clean['feature_1'] = df_clean['feature_1'].replace([np.inf, -np.inf], np.nan)
median_val = df_clean['feature_1'].median()
df_clean['feature_1'] = df_clean['feature_1'].fillna(median_val)

In [23]:
# 检查是否还有异常值
np.isinf(df_clean.select_dtypes(include=['float64', 'int64'])).sum()

age               0
campaign          0
pdays             0
previous          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
feature_1         0
feature_2         0
y                 0
dtype: int64

In [13]:
# 检查是否还有缺失值
df_clean.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
feature_1         0
feature_2         0
feature_3         0
feature_4         0
feature_5         0
y                 0
dtype: int64

In [24]:
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
categorical_cols

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome',
 'feature_3',
 'feature_4',
 'feature_5']

In [25]:
df_tree_ready = df_clean.copy()
df_tree_ready.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,feature_1,feature_2,feature_3,feature_4,feature_5,y
0,72,retired,married,basic.4y,no,no,no,telephone,apr,tue,...,93.075,-47.1,1.453,5099.1,1.3874,73.8872,vg1,lv1,fn1,0
1,30,admin.,single,university.degree,no,yes,no,cellular,aug,tue,...,94.027,-38.3,0.886,4991.6,1.3874,31.8872,vg2,lv1,fn2,0
2,31,unemployed,married,university.degree,no,yes,no,cellular,aug,fri,...,93.444,-36.1,4.966,5228.1,4.3904,35.8872,vg3,lv1,fn3,0
3,37,admin.,married,high.school,no,yes,yes,cellular,nov,mon,...,92.649,-30.1,0.722,5017.5,1.3874,38.8872,vg4,lv1,fn4,0
4,53,unemployed,divorced,basic.9y,unknown,no,no,telephone,may,thu,...,93.994,-36.4,4.860,5191.0,2.3884,55.8872,vg5,lv1,fn5,0


### 对训练集和测试集分别使用5-fold交叉验证的Target Encoding
Target Encoding核心思想是用目标变量的平均值来替换分类变量的每个类别。
比如有marital分为divorced, married, single, unknown四种，那每种的编码就是每种的出现次数占总行数的比例。
考虑到减少数据集本身的影响，使用5折交叉验证来计算平均比例，也就是编码。
Target Encoding用于Random Forest

In [ ]:
df_test = pd.read_csv("../Data/bank_marketing_test.csv")
df_test['pdays'] = df_test['pdays'].replace(999,-1)
df_test['feature_1'] = df_test['feature_1'].replace([np.inf, -np.inf], np.nan)
median_val = df_test['feature_1'].median()
df_test['feature_1'] = df_test['feature_1'].fillna(median_val)

In [ ]:
from sklearn.model_selection import KFold

# ==========================================================
# 阶段 1：“学习” - 仅从 df_tree_ready (训练集) 中学习
# ==========================================================
print("--- 阶段 1：正在从训练集学习全局编码字典 (用于测试集)... ---")

# 1.1 计算“安全网”：全局订阅率 (用于测试集的新类别)
global_mean = df_tree_ready['y'].mean()

# 1.2 创建一个字典来存储所有“func”
# 这个 'encoding_maps' 将 *只* 用于测试集
encoding_maps = {} 

for col in categorical_cols:
    func = df_tree_ready.groupby(col)['y'].mean()
    encoding_maps[col] = func
    print(f"已学习 '{col}' 的 *全局* 编码规则。")

print(f"已学习“全局均值”: {global_mean:.4f}")


# ==========================================================
# 阶段 2：“应用” 
# ==========================================================

# 2.1 应用到 df_test (测试集)
print("\n--- 阶段 2.1：正在转换 df_test (测试集)... ---")
for col in categorical_cols:
    if col not in df_test.columns: continue
        
    original_index = df_test.columns.get_loc(col)
    func = encoding_maps[col] # 使用“全局”字典
    encoded_data = df_test[col].map(func)
    encoded_data = encoded_data.fillna(global_mean) # 缺失值按全局均值代替
    
    df_test.pop(col)
    df_test.insert(original_index, col, encoded_data)
print("--- 测试集转换完成 ---")
print(df_test.head())


# 2.2 应用到 df_tree_ready (训练集）(使用 K-Fold 交叉验证来防止数据泄露)
print("\n--- 阶段 2.2：正在对 df_tree_ready (训练集) 执行 K-Fold 编码... ---")

# 5折交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for col in categorical_cols:
    # 1. 找到原始列的位置
    original_index = df_tree_ready.columns.get_loc(col)
    
    # 2. 创建一个临时的空 Series，用于存储“无泄露”的编码结果
    encoded_series = pd.Series(index=df_tree_ready.index, dtype=float, name=f"{col}_encoded")
    
    # 3. 遍历 K-Fold
    for train_idx, val_idx in kf.split(df_tree_ready):
        # 3a. 将数据分为“学习集”(train_fold)和“编码集”(val_fold)
        df_train_fold = df_tree_ready.iloc[train_idx]
        df_val_fold = df_tree_ready.iloc[val_idx]
        
        # 3b. 只在“训练集”(train_fold)上计算均值
        func_fold = df_train_fold.groupby(col)['y'].mean()
        
        # 3c. 将均值应用到“编码集”(val_fold)上
        val_mapped = df_val_fold[col].map(func_fold)
        
        # 3d. 将结果存入临时 Series
        encoded_series.iloc[val_idx] = val_mapped

    # 4. 处理训练集中的罕见类别
    # (如果某个类别只在 1 折中出现，它在其他 4 折中会被编码为 NaN)
    # 用“全局均值”来填充这些罕见类别
    encoded_series = encoded_series.fillna(global_mean)
    
    # 5. 替换原始列，保证原始顺序
    df_tree_ready.pop(col)
    df_tree_ready.insert(original_index, col, encoded_series)
    
print("--- 训练集 K-Fold 编码完成 ---")
print(df_tree_ready.head())

--- 阶段 1：正在从训练集学习 *全局* 编码字典 (用于测试集)... ---
已学习 'job' 的 *全局* 编码规则。
已学习 'marital' 的 *全局* 编码规则。
已学习 'education' 的 *全局* 编码规则。
已学习 'default' 的 *全局* 编码规则。
已学习 'housing' 的 *全局* 编码规则。
已学习 'loan' 的 *全局* 编码规则。
已学习 'contact' 的 *全局* 编码规则。
已学习 'month' 的 *全局* 编码规则。
已学习 'day_of_week' 的 *全局* 编码规则。
已学习 'poutcome' 的 *全局* 编码规则。
已学习 'feature_3' 的 *全局* 编码规则。
已学习 'feature_4' 的 *全局* 编码规则。
已学习 'feature_5' 的 *全局* 编码规则。
已学习“全局均值”: 0.1190

--- 阶段 2.1：正在转换 df_test (测试集)... ---
--- 测试集转换完成 ---
   age       job   marital  education   default   housing      loan   contact  \
0   34  0.072348  0.107899   0.083115  0.136194  0.114267  0.120097  0.052679   
1   29  0.140991  0.107899   0.114754  0.136194  0.123339  0.120097  0.052679   
2   38  0.104885  0.107899   0.114754  0.054755  0.114267  0.120097  0.052679   
3   31  0.081851  0.107899   0.114754  0.054755  0.114267  0.120097  0.052679   
4   45  0.072348  0.107899   0.121739  0.136194  0.123339  0.120097  0.052679   

      month  day_of_week  ...  emp.var.rate

### 查看编码完的训练集和测试集

In [28]:
df_tree_ready.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,feature_1,feature_2,feature_3,feature_4,feature_5,y
0,72,0.276094,0.106673,0.111006,0.135762,0.113094,0.120241,0.052882,0.197097,0.127551,...,93.075,-47.1,1.453,5099.1,1.3874,73.8872,0.147414,0.093178,0.312903,0
1,30,0.139535,0.145894,0.142951,0.135176,0.122847,0.119408,0.158305,0.118536,0.124086,...,94.027,-38.3,0.886,4991.6,1.3874,31.8872,0.187628,0.091879,0.153200,0
2,31,0.145191,0.106548,0.142951,0.135176,0.122847,0.119408,0.158305,0.118536,0.113981,...,93.444,-36.1,4.966,5228.1,4.3904,35.8872,0.103683,0.091879,0.170213,0
3,37,0.137885,0.106673,0.116769,0.135762,0.123829,0.111929,0.159330,0.110270,0.106064,...,92.649,-30.1,0.722,5017.5,1.3874,38.8872,0.087690,0.093178,0.126443,0
4,53,0.143389,0.107055,0.078457,0.053732,0.115073,0.121201,0.053369,0.067419,0.129515,...,93.994,-36.4,4.860,5191.0,2.3884,55.8872,0.143698,0.093524,0.153061,0


In [29]:
df_test.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,feature_1,feature_2,feature_3,feature_4,feature_5
0,34,0.072348,0.107899,0.083115,0.136194,0.114267,0.120097,0.052679,0.065951,0.104064,...,1.1,93.994,-36.4,4.857,5191.0,4.6443,38.8889,0.102041,0.093140,0.078288
1,29,0.140991,0.107899,0.114754,0.136194,0.123339,0.120097,0.052679,0.107835,0.112035,...,1.4,94.465,-41.8,4.959,5228.1,3.6433,32.8889,0.104371,0.093140,0.123832
2,38,0.104885,0.107899,0.114754,0.054755,0.114267,0.120097,0.052679,0.065951,0.127534,...,1.1,93.994,-36.4,4.860,5191.0,4.6443,42.8889,0.086483,0.093140,0.075630
3,31,0.081851,0.107899,0.114754,0.054755,0.114267,0.120097,0.052679,0.107835,0.127534,...,1.4,94.465,-41.8,4.866,5228.1,2.6423,33.8889,0.102041,0.093140,0.075676
4,45,0.072348,0.107899,0.121739,0.136194,0.123339,0.120097,0.052679,0.065951,0.127534,...,-1.8,93.876,-40.0,0.683,5008.7,2.6423,NaN,0.083591,0.859155,0.083045


### 标准化数据，用于logistic regression，SVM，决策树

In [30]:
df_lrsvm_ready = df_tree_ready.copy()
categorical_cols

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome',
 'feature_3',
 'feature_4',
 'feature_5']

In [31]:
# 将所有 bool 类型的列转为 float，提高模型兼容性
bool_cols = df_lrsvm_ready.select_dtypes(include=['bool']).columns
df_lrsvm_ready[bool_cols] = df_lrsvm_ready[bool_cols].astype(float)

df_lrsvm_ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26246 entries, 0 to 26245
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             26246 non-null  int64  
 1   job             26246 non-null  float64
 2   marital         26246 non-null  float64
 3   education       26246 non-null  float64
 4   default         26246 non-null  float64
 5   housing         26246 non-null  float64
 6   loan            26246 non-null  float64
 7   contact         26246 non-null  float64
 8   month           26246 non-null  float64
 9   day_of_week     26246 non-null  float64
 10  campaign        26246 non-null  int64  
 11  pdays           26246 non-null  int64  
 12  previous        26246 non-null  int64  
 13  poutcome        26246 non-null  float64
 14  emp.var.rate    26246 non-null  float64
 15  cons.price.idx  26246 non-null  float64
 16  cons.conf.idx   26246 non-null  float64
 17  euribor3m       26246 non-null 

In [32]:
df_lrsvm_ready.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,feature_1,feature_2,feature_3,feature_4,feature_5,y
0,72,0.276094,0.106673,0.111006,0.135762,0.113094,0.120241,0.052882,0.197097,0.127551,...,93.075,-47.1,1.453,5099.1,1.3874,73.8872,0.147414,0.093178,0.312903,0
1,30,0.139535,0.145894,0.142951,0.135176,0.122847,0.119408,0.158305,0.118536,0.124086,...,94.027,-38.3,0.886,4991.6,1.3874,31.8872,0.187628,0.091879,0.153200,0
2,31,0.145191,0.106548,0.142951,0.135176,0.122847,0.119408,0.158305,0.118536,0.113981,...,93.444,-36.1,4.966,5228.1,4.3904,35.8872,0.103683,0.091879,0.170213,0
3,37,0.137885,0.106673,0.116769,0.135762,0.123829,0.111929,0.159330,0.110270,0.106064,...,92.649,-30.1,0.722,5017.5,1.3874,38.8872,0.087690,0.093178,0.126443,0
4,53,0.143389,0.107055,0.078457,0.053732,0.115073,0.121201,0.053369,0.067419,0.129515,...,93.994,-36.4,4.860,5191.0,2.3884,55.8872,0.143698,0.093524,0.153061,0


In [33]:
# 1. y 是标签列
y_lrsvm = df_lrsvm_ready['y']

# 2. X 是除了 y 之外的所有特征
X_lrsvm = df_lrsvm_ready.drop(columns=['y'])

# 查看一下结果
X_lrsvm.shape, y_lrsvm.shape

((26246, 24), (26246,))

In [ ]:
from sklearn.preprocessing import StandardScaler
# 1. 创建 StandardScaler 实例
#    参数说明：
#    - with_mean=True：对每列减去均值（默认）
#    - with_std=True：除以标准差（默认）
#    这是最常用的 Z-score 标准化
scaler = StandardScaler()

# 2. 使用训练集拟合 scaler（学习每一列的 mean 和 std）
#    注意：这里不能对 test 数据 fit！否则数据泄露。
scaler.fit(X_lrsvm)

# 3. 对训练集进行标准化，把数值缩放到 N(0,1)
X_lrsvm_scaled = scaler.transform(X_lrsvm)

# 4. transform 输出是 numpy array，我们转回 DataFrame，保持列名一致
X_lrsvm_scaled = pd.DataFrame(
    X_lrsvm_scaled,
    columns=X_lrsvm.columns,
    index=X_lrsvm.index
)

# 查看形状确认
X_lrsvm_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26246 entries, 0 to 26245
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             26246 non-null  float64
 1   job             26246 non-null  float64
 2   marital         26246 non-null  float64
 3   education       26246 non-null  float64
 4   default         26246 non-null  float64
 5   housing         26246 non-null  float64
 6   loan            26246 non-null  float64
 7   contact         26246 non-null  float64
 8   month           26246 non-null  float64
 9   day_of_week     26246 non-null  float64
 10  campaign        26246 non-null  float64
 11  pdays           26246 non-null  float64
 12  previous        26246 non-null  float64
 13  poutcome        26246 non-null  float64
 14  emp.var.rate    26246 non-null  float64
 15  cons.price.idx  26246 non-null  float64
 16  cons.conf.idx   26246 non-null  float64
 17  euribor3m       26246 non-null 

In [ ]:
# 保存数据
df_tree_ready.to_csv("Data/tree_data_clean.csv", index=False)
df_test.to_csv("Data/tree_data_clean_test.csv", index=False)

In [36]:
X_lrsvm_scaled.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,feature_1,feature_2,feature_3,feature_4,feature_5
0,3.047313,3.010164,-0.674788,-0.318808,0.504554,-1.218759,0.419233,-1.278002,0.867855,0.889983,...,-1.178334,-0.863131,-1.422322,-1.229436,-0.907371,-0.156530,2.781528,0.794919,-0.227453,3.205529
1,-0.971694,0.394070,1.483801,0.961275,0.486945,0.807553,0.145058,0.761206,-0.004823,0.531515,...,-1.114969,0.761336,0.471641,-1.554560,-2.375493,-0.156530,-1.089421,1.919378,-0.239260,0.565752
2,-0.876004,0.502417,-0.681651,0.961275,0.486945,0.807553,0.145058,0.761206,-0.004823,-0.513842,...,0.849346,-0.233480,0.945131,0.784956,0.854376,-0.130636,-0.720760,-0.427871,-0.239260,0.846954
3,-0.301860,0.362465,-0.674788,-0.087868,0.504554,1.011622,-2.316321,0.781029,-0.096646,-1.332793,...,-2.192174,-1.590046,2.236469,-1.648600,-2.021778,-0.156530,-0.444263,-0.875070,-0.227453,0.123477
4,1.229190,0.467908,-0.653764,-1.623081,-1.959220,-0.807579,0.735234,-1.268593,-0.572646,1.093111,...,0.659251,0.705025,0.880564,0.724174,0.347703,-0.147898,1.122550,0.691032,-0.224308,0.563451


In [ ]:
# 保存标准化后的特征矩阵
X_lrsvm_scaled.to_csv("Data/lrsvm_X_scaled.csv", index=False)
# 保存 y
y_lrsvm.to_csv("Data/lrsvm_y.csv", index=False)